This notebook aims to find out electrodes under specific criterion.

# Criterion 1: visual responded during study phase

Find electrodes have visual response (i.e. mean NBG activity > 50%) during the study phase within visual cortex.

In [1]:
import os
import numpy as np
from ECoGBasic import SpectralBasic
from functools import reduce
import mat73

In [2]:
# subject info
sbj_ls = ['YBG','YBI','YBJ','YBN','YCP']

recall_blk_dict = {
          'YBG' : ['007','008','009'],
          'YBI' : ['014'],
          'YBJ' : ['010'],
          'YBN' : ['038', '039', '040'],
          'YCP' : ['038'] # block 039 of YCP doesn't have study phase
          }

In [3]:
# home directory
home_path = '~/Documents/MATLAB/ECoG'
home_dir  = os.path.expanduser(home_path)

In [4]:
# initiate all basic object
basic_recall_dict = {}

for sj in sbj_ls:
    basic_recall_obj = SpectralBasic(sbj_name = sj,
                               blk_name = recall_blk_dict[sj],
                               task_name = 'vis_contrast_recall')
    
    basic_recall_dict[sj] = basic_recall_obj

In [5]:
# get visual electrodes
elec_vis_dict = {}

for sj in sbj_ls:
    basic_recall_obj = basic_recall_dict[sj]
        
    elec_vis = basic_recall_obj.select_channel() 
    
    # save the good visual electrodes to dict
    elec_vis_dict[sj] = elec_vis
    
    # update object dict (select_channel() will get master_dict and event_dict to object)
    basic_recall_dict[sj] = basic_recall_obj

In [6]:
# epoch window

# All subjects are using the standard recording setting, here I manually code the parameters.
# Codes need to be rewritten into more general way if using different recording settings among subjects.

# tf_srate = round(master_vars.ecog_srate/master_vars.compress);
tf_srate = round(2000/2)

study_epoch_prestim  = np.floor(1*tf_srate)
study_epoch_poststim = np.ceil(1*tf_srate)
epoch_norm           = np.floor(0.5*tf_srate)

In [46]:
def apply_criterion(sj,ci):
    
    # (Part1) Get trial-wise spectrum
    trial_index = 0 # index for looping through trials of all blocks in a task
    
    nblk = len(recall_blk_dict[sj])
    
    # layer 1: loop through all blocks
    for iblk in range(nblk):
        blk = recall_blk_dict[sj][iblk]

        # epoch trigger
        study_onsets = basic_recall_dict[sj].event_dict[blk]['study_tone_trial_onset'][0][0]
        study_trigger = np.round(study_onsets/2) # master_vars.compress=2

        # extract spectral data
        Spect_dir = home_dir + '/neuralData/SpecData'
        Spect_path = Spect_dir + '/{}/{}/TF_decomp_CAR_{}_{}_{}.mat'.format(sj,blk,sj,blk,ci)

        tf_band = mat73.loadmat(Spect_path)['band']
        tf_amp  = tf_band['tf_data']['amplitude'].T

        # layer 2: loop through trials
        ntrial = len(study_trigger[0])
        
        for i in range(len(study_trigger[0])):
            
            epoch_start = (study_trigger[0,i] - study_epoch_prestim - 1).astype(int)
            epoch_end  = (study_trigger[0,i] + study_epoch_poststim).astype(int)
            tmp_epoch = tf_amp[:,epoch_start:epoch_end]
            
           
            # normalization
            norm_start = (study_epoch_prestim - epoch_norm-1).astype(int)
            norm_end   = (study_epoch_prestim).astype(int)
            norm_vect  = np.mean(tmp_epoch[:,norm_start:norm_end],axis = 1)
                
            # normalization
            norm_vect  = norm_vect.reshape([len(norm_vect),1])
            tmp_normed = tmp_epoch/norm_vect
            
            # paste result
            tmp_normed = np.expand_dims(tmp_normed, axis = 0)
            if trial_index == 0:
                spectral_mx = tmp_normed
            else:
                spectral_mx = np.append(spectral_mx, tmp_normed, axis = 0)
                    
            trial_index += 1
    
    # Spectrum of all trials were stored in 'spectral_mx' (trial number x frequencies x time points)
    
    # (Part 2) Get mean spectrum of 250~500ms of study phase time window
    freq_vect = np.arange(2,204,2)
    NBG_start = np.where(freq_vect == 20)[0][0]
    NBG_end   = np.where(freq_vect == 60)[0][0] + 1 
    
    vis_response = np.mean(spectral_mx[:,NBG_start:NBG_end,1500:-1])
    
    # (Part 3) Compare with 50%
    return vis_response >= 50

In [47]:
# empty result dict
elect_vis_response_dict = {
    'YBG':[],
    'YBI':[],
    'YBJ':[],
    'YBN':[],
    'YCP':[],
}

for sj in sbj_ls:
    vis_elec = elec_vis_dict[sj]
    
    for ci in vis_elec:
        int_ci = int(ci)
        if apply_criterion(sj,int_ci):
            elect_vis_response_dict[sj].append(int_ci)

In [42]:
len(elec_vis_dict['YBN'])

31

In [44]:
len(elect_vis_response_dict['YBN'])

0

In [9]:
percent = [round(len(elect_vis_response_dict[sj])/len(elec_vis_dict[sj]),2) for sj in sbj_ls]

In [45]:
elect_vis_response_dict

{'YBG': [], 'YBI': [], 'YBJ': [], 'YBN': [], 'YCP': []}

In [10]:
for sj in sbj_ls:
    print(sj)
    print('Number of visual response electrodes: {}'.format(len(elect_)))

[1.0, 1.0, 1.0, 1.0, 1.0]

In [28]:
23/25

0.92